In [2]:
import string
import re

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nltk.download('stopwords')
STOPWORDS_AND_CHARS = stopwords.words('russian')

COUNT_RUSSIAN_LETTERS = 33
RUSSIAN_ALPHABET = [chr(0x0410 + index) for index in range(COUNT_RUSSIAN_LETTERS)]
RUSSIAN_ALPHABET.extend([chr(0x0430 + index) for index in range(COUNT_RUSSIAN_LETTERS)])
STOPWORDS_AND_CHARS.extend(string.punctuation)
STOPWORDS_AND_CHARS.extend(RUSSIAN_ALPHABET)

emoji_finder = re.compile('[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]+')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maksimmigur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка данных

In [4]:
bot_posts_df = pd.read_excel('./Data/ai_bot_app_post.xls')
posts_df = pd.read_excel('./Data/posts (1).xlsx')

## Информация о наборах данных

In [5]:
bot_posts_df.head()

,user_id,telegram_id,text,title
0,1,157314,❗️Восстановление аммиакопровода Тольятти — Оде...,Восстановление аммиакопровода займет до 3 меся...
1,2,18940,Дополнительные 39 миллионов рублей были выделе...,Выделены дополнительные 39 млн рублей на соцуч...
2,3,4868,«Россия уничтожена санкциями»\n ...,«Россия уничтожена санкциями»\n ...
3,4,157315,❗️Правоохранители провели обыски в министерств...,Обыски в минобразования Дагестана по делу о вы...
4,16,157317,"Реконструкция пропускного пункта ""Верхний Ларс...","Реконструкция пропускного пункта ""Верхний Ларс..."


In [6]:
posts_df.head()

,id,telegram_id,text,datetime,category_id,channel_id,title,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,1,157314,❗️Восстановление аммиакопровода Тольятти — Оде...,2023-06-07 11:40:06,NaN,-1001394050290,Восстановление аммиакопровода займет до 3 меся...,NaN,NaN,NaN
1,2,18940,Дополнительные 39 миллионов рублей были выделе...,2023-06-07 11:34:56,NaN,-1001098860759,Выделены дополнительные 39 млн рублей на соцуч...,NaN,NaN,NaN
2,3,4868,«Россия уничтожена санкциями»\n ...,2023-06-07 11:03:05,NaN,-1001797434593,«Россия уничтожена санкциями»\n ...,NaN,NaN,NaN
3,4,157315,❗️Правоохранители провели обыски в министерств...,2023-06-07 11:50:06,NaN,-1001394050290,Обыски в минобразования Дагестана по делу о вы...,NaN,NaN,NaN
4,16,157317,"Реконструкция пропускного пункта ""Верхний Ларс...",2023-06-07 12:10:25,NaN,-1001394050290,"Реконструкция пропускного пункта ""Верхний Ларс...",NaN,NaN,NaN


In [7]:
bot_posts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88182 entries, 0 to 88181
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88182 non-null  int64 
 1   telegram_id  88182 non-null  int64 
 2   text         88178 non-null  object
 3   title        88177 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.7+ MB


In [8]:
posts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169881 entries, 0 to 169880
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           169881 non-null  object
 1   telegram_id  169880 non-null  object
 2   text         169871 non-null  object
 3   datetime     169871 non-null  object
 4   category_id  21 non-null      object
 5   channel_id   169863 non-null  object
 6   title        169848 non-null  object
 7   Unnamed: 7   2 non-null       object
 8   Unnamed: 8   2 non-null       object
 9   Unnamed: 9   1 non-null       object
dtypes: object(10)
memory usage: 13.0+ MB


## Предобработка текста

Этапы обработки текста:
* Удаление символов(пунктуации, буквы которые в предложение стоят по одиночке).
* Удаление стоп слов
* Приведение слов к номральной форме
* Токетизация
* Векторизация

Удалим не нужные столбцы, которые не имею смысла.

In [9]:
posts_df = posts_df[[
    'id', 
    'telegram_id', 
    'text', 'datetime', 
    'channel_id', 
    'title']
]
posts_df['text'] = posts_df['text'].apply(str)

Напишем функции для обработки текстов.

In [10]:
def remove_shit(news: str) -> str:
    without_emojis_n_tabs = re.sub(emoji_finder, '', news).replace('\n', '').replace('\xa0', '')
    without_links = re.sub(r'^https?:\/\/.*[\r\n]*', '', without_emojis_n_tabs, flags=re.MULTILINE)

    without_digits = (
        ''.join([word for word in without_links if (word not in string.punctuation) and (word not in string.digits)])
    )
    news = ' '.join([word for word in without_digits.split(' ') if word not in STOPWORDS_AND_CHARS])
    return news

def lemmatize(news: str) -> str:
    pymorphy2_analyzer = MorphAnalyzer()
    news = ' '.join([pymorphy2_analyzer.parse(word)[0].normal_form.strip() for word in news.split(' ')]).strip()
    return news

def tokenize(news: str) -> list:
    news = sent_tokenize(' '.join([word.strip() for word in news.split(' ')]))
    return news

def vectorize(news: str) -> np.array:
    tf_idf_model = TfidfVectorizer().fit_transform(news)
    return tf_idf_model.toarray()

## Построение модели схожести новостей
Для того, чтобы найти тексты, близкие друг другу не только на уровне похожести строк, но и по смыслу, можно использовать векторные представления текста.
Идея следующая:

* кодируем текст при помощи понравившейся модели (word2vec, fastText или Sentence Transformers, например, LABSE);
* вычисляем близость между получившимися векторами при помощи, например, косинусной меры сходства;
* ранжируем тексты по схожести и, в зависимости от задачи, определяем подходящий порог, после которого данные будут отфильтровываться;
удаляем слишком близкие друг к другу тексты.

> Вывод: мы сделали векторизацию новостей сравнили их всех попарно и нашли оптимальный порого при котором можно удалять дубликаты.

In [21]:
import json
import csv


with open('./Data/cats.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=' ', quotechar='|')
    rows = [json.loads(row[0].replace("'", '"')) for row in reader]
    

JSONDecodeError: Expecting ',' delimiter: line 1 column 45 (char 44)